In [22]:
import ot
import importing
import numpy as np

train = [76,124,130,154,157,174,383,501,524,593,711,732,798,800,959,987]
test = [1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,1100,1101,1102,1103,1104,1105,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115,1116,1117,1118,1119,1120,1121,1122,1123,1124,1125,1126,1127,1128,1129,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179,1180,1181,1182,1183,1184,1185,1186,1187,1188,1189,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199]
blades = importing.blades2(ids = train)
blade1 = blades[1, :, :]
blade2 = blades[2, :, :]

In [6]:
distances = np.zeros((blades.shape[0], blades.shape[0]))
for i, x1 in enumerate(blades):
    print(x1.shape)
    for j, x2 in enumerate(blades):
        distances[i, j] = ot.sliced_wasserstein_distance(X_s = x1, X_t = x2, 
                               a = None, b = None, 
                               n_projections = 100, p = 3, 
                               projections=None, seed=None, log=False)


(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)
(29773, 3)


In [20]:
import GPy
x_scalars, y_scalars = importing.scalars(train, test = False)
blades = blades.reshape(16, -1)
kernel_sinkhorn = GPy.kern.Matern52(input_dim = 29773 * 3, ARD = False)
custom_kernel_matrix = kernel_sinkhorn.K_of_r(distances)
custom_kernel_gradient = kernel_sinkhorn.dK_dr(distances)

# Create GP regression model with the custom kernel
gp_model = GPy.models.GPRegression(blades, y_scalars[:, 0:1], kernel = kernel_sinkhorn)

# Now you can optimize the hyperparameters of the GP model
gp_model.optimize_restarts(num_restarts = 10, messages = False, max_iters = 5000)

Optimization restart 1/10, f = -39.47764514784494
Optimization restart 2/10, f = -51.0732901863425
Optimization restart 3/10, f = -51.07326469037306
Optimization restart 4/10, f = -51.07328141592652
Optimization restart 5/10, f = -51.07325081350168
Optimization restart 6/10, f = -39.47764538559618
Optimization restart 7/10, f = -39.47765289129351
Optimization restart 8/10, f = -51.073290148702675
Optimization restart 9/10, f = -51.07322287674683
Optimization restart 10/10, f = -39.47764227995366


In [26]:
blades_test = importing.blades2(ids = test)
blades_test = blades_test.reshape(200, -1)

x_scalars_test, y_scalars_test = importing.scalars(test, test = True)

y_pred, _ = gp_model.predict(blades_test)


In [27]:
from packages import *
q2 = r2_score(y_scalars_test[:, 0:1], y_pred)


In [28]:
q2

-0.02557894999550303